In [1]:
import sys
sys.path.append('/home/onyxia/work/libsigma')

In [6]:
import os
import read_and_write as rw
import gdal

ModuleNotFoundError: No module named 'gdal'

# Partie production image

L'idée ça peut être de construire une image comme dans le td opération sur raster en lui ajoutant à chaque fois une nouvelle bande d'une des dates

Donc d'abord charger toutes les images dans un tableau avec 60 bandes

In [14]:
ds_test=rw.open_image('/home/onyxia/work/data/images/SENTINEL2B_20220125-105852-948_L2A_T31TCJ_C_V3-0_SRE_B2.tif')

In [4]:
img1=rw.load_img_as_array('/home/onyxia/work/data/images/SENTINEL2B_20220125-105852-948_L2A_T31TCJ_C_V3-0_SRE_B2.tif')

In [5]:
img1

array([[[3122],
        [3112],
        [3037],
        ...,
        [ 441],
        [ 363],
        [ 315]],

       [[3228],
        [3293],
        [3220],
        ...,
        [ 421],
        [ 331],
        [ 297]],

       [[3245],
        [3348],
        [3306],
        ...,
        [ 337],
        [ 321],
        [ 336]],

       ...,

       [[ 224],
        [ 258],
        [ 275],
        ...,
        [ 835],
        [ 800],
        [ 874]],

       [[ 243],
        [ 269],
        [ 269],
        ...,
        [ 844],
        [ 828],
        [ 897]],

       [[ 189],
        [ 263],
        [ 295],
        ...,
        [ 783],
        [ 795],
        [ 839]]], dtype=int16)